In [311]:
import torch
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from transformers import TrainingArguments, Trainer
# import transformers
# from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
# from transformers import AutoTokenizer, BertTokenizer, BertModel, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
# from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler

In [312]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

# Download

In [313]:
url = './radioDatabaseRecordClean.csv'

In [314]:
df = pd.read_csv(url, sep='|')
df

,Id,Date,Report,Type
0,0,2022-07-13 02:57:59.247,EMERGENCY MDCT OF THE BRAIN HISTORY: Known c...,CT
1,1,2022-07-13 06:32:58.923,"MRI OF THE BRAIN \t History: U/D DM, HT, DLP,...",MRI
2,2,2022-07-13 06:35:10.970,MRI OF THE BRAIN AND HIPPOCAMPI (dementia prot...,MRI
3,2,2022-07-13 06:35:57.020,MRI OF THE BRAIN AND HIPPOCAMPI (dementia prot...,MRI
4,3,2022-07-13 07:28:35.153,MRI OF THE PARANASAL SINUSES AND NECK HISTOR...,MRI
...,...,...,...,...
30886,19388,2023-07-17 13:33:05.590,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
30887,19388,2023-07-17 13:33:05.590,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
30888,19388,2023-07-17 13:33:05.590,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
30889,19388,2023-07-17 14:10:19.167,EMERGENCY MRI OF THE BRAIN (limited protocol f...,MRI


In [315]:
df.duplicated().sum(), df['Report'].duplicated().sum()

(10965, 11545)

In [316]:
df.drop_duplicates()

,Id,Date,Report,Type
0,0,2022-07-13 02:57:59.247,EMERGENCY MDCT OF THE BRAIN HISTORY: Known c...,CT
1,1,2022-07-13 06:32:58.923,"MRI OF THE BRAIN \t History: U/D DM, HT, DLP,...",MRI
2,2,2022-07-13 06:35:10.970,MRI OF THE BRAIN AND HIPPOCAMPI (dementia prot...,MRI
3,2,2022-07-13 06:35:57.020,MRI OF THE BRAIN AND HIPPOCAMPI (dementia prot...,MRI
4,3,2022-07-13 07:28:35.153,MRI OF THE PARANASAL SINUSES AND NECK HISTOR...,MRI
...,...,...,...,...
30884,19386,2023-07-17 12:09:10.360,MRI OF THE BRAIN HISTORY: Known case of rela...,MRI
30885,19387,2023-07-17 15:17:17.250,MDCT OF THE BRAIN HISTORY: Acute right PCA t...,CT
30886,19388,2023-07-17 13:33:05.590,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
30889,19388,2023-07-17 14:10:19.167,EMERGENCY MRI OF THE BRAIN (limited protocol f...,MRI


In [317]:
df.drop_duplicates(subset=['Report'])

,Id,Date,Report,Type
0,0,2022-07-13 02:57:59.247,EMERGENCY MDCT OF THE BRAIN HISTORY: Known c...,CT
1,1,2022-07-13 06:32:58.923,"MRI OF THE BRAIN \t History: U/D DM, HT, DLP,...",MRI
2,2,2022-07-13 06:35:10.970,MRI OF THE BRAIN AND HIPPOCAMPI (dementia prot...,MRI
4,3,2022-07-13 07:28:35.153,MRI OF THE PARANASAL SINUSES AND NECK HISTOR...,MRI
6,4,2022-07-13 06:54:12.140,MDCT OF THE BRAIN HISTORY: F/U a of NPH S/P ...,CT
...,...,...,...,...
30884,19386,2023-07-17 12:09:10.360,MRI OF THE BRAIN HISTORY: Known case of rela...,MRI
30885,19387,2023-07-17 15:17:17.250,MDCT OF THE BRAIN HISTORY: Acute right PCA t...,CT
30886,19388,2023-07-17 13:33:05.590,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
30889,19388,2023-07-17 14:10:19.167,EMERGENCY MRI OF THE BRAIN (limited protocol f...,MRI


In [318]:
df.drop_duplicates(subset=['Report'], inplace=True)
df.duplicated().sum(), df['Report'].duplicated().sum()

(0, 0)

In [319]:
# sort by Id, Date
df.sort_values(by=['Id','Date'], inplace=True)
df

,Id,Date,Report,Type
0,0,2022-07-13 02:57:59.247,EMERGENCY MDCT OF THE BRAIN HISTORY: Known c...,CT
1,1,2022-07-13 06:32:58.923,"MRI OF THE BRAIN \t History: U/D DM, HT, DLP,...",MRI
2,2,2022-07-13 06:35:10.970,MRI OF THE BRAIN AND HIPPOCAMPI (dementia prot...,MRI
4,3,2022-07-13 07:28:35.153,MRI OF THE PARANASAL SINUSES AND NECK HISTOR...,MRI
6,4,2022-07-13 06:54:12.140,MDCT OF THE BRAIN HISTORY: F/U a of NPH S/P ...,CT
...,...,...,...,...
30884,19386,2023-07-17 12:09:10.360,MRI OF THE BRAIN HISTORY: Known case of rela...,MRI
30885,19387,2023-07-17 15:17:17.250,MDCT OF THE BRAIN HISTORY: Acute right PCA t...,CT
30886,19388,2023-07-17 13:33:05.590,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
30889,19388,2023-07-17 14:10:19.167,EMERGENCY MRI OF THE BRAIN (limited protocol f...,MRI


In [320]:
df2 = df
df2.sample(5)

,Id,Date,Report,Type
2355,1523,2022-08-12 07:17:14.983,MRI AND MRA OF THE BRAIN HISTORY: Chronic head...,MRI
17687,11153,2023-02-12 10:47:03.840,MRI AND MRA OF THE BRAIN (RS PROTOCOL) Histo...,MRI
19433,12223,2023-03-03 06:55:16.497,MRI OF THE BRAIN \t History: Memory complaint...,MRI
8642,5460,2022-10-29 18:39:44.707,"MRI, MRA OF THE BRAIN AND IAC History:\tSynco...",MRI
30235,18969,2023-07-08 16:13:29.037,MRI LUMBOSACRAL AND SCREENING WHOLE SPINE His...,MRI


# DisplayRegex

In [321]:
# Version 1
# def printReport(df, column, idx):
#     text = df[column][idx].split('    ')
#     print(df.shape)
#     print('\n\n'.join(text))

In [322]:
print("\x1b[43m\x1b[31m\x1B[1m\"red\"\x1b[0m other text")
# print("AB\x1b[43mCD\x1b[35mEF\x1b[1mGH\x1b[4mIJ\x1b[7m"
#       "KL\x1b[49mMN\x1b[39mOP\x1b[22mQR\x1b[24mST\x1b[27mUV")

"red" other text


In [323]:
import re

def highlightText(s,term):
    colourFormat = '\033[{0}m'
    colourStr = colourFormat.format(32)
    resetStr = colourFormat.format(0)
    colourStr = '\x1b[40m\x1b[32m\x1B[1m'
    resetStr = '\x1b[0m'
    # s = "This is a sentence where I talk about interesting stuff like sencha tea."

    lastMatch = 0
    formattedText = ''
    my_regex = r"" + term + r"*\w+"
    for match in re.finditer(my_regex, s, re.IGNORECASE):
        start, end = match.span()
        formattedText += s[lastMatch: start]
        formattedText += colourStr
        formattedText += s[start: end]
        formattedText += resetStr
        lastMatch = end
    formattedText += s[lastMatch:]
    # for multiple \n new lines
    # formattedText = re.sub(r'\n{2}', '', formattedText)
    print(formattedText)

highlightText("Sen This is a sentence where I talk about interesting stuff like sencha tea.", 'talk|sen')

Sen This is a sentence where I talk about interesting stuff like sencha tea.


In [324]:
# Version 2
# def printReport(df, column, idx, term= ''):
#     text = df[column][idx].split('   ')
#     # text = [word.strip() for word in df[column][idx].split('  ')]
#     print(df.shape)
#     highlightText('\n\n'.join(text), term)

In [325]:
def printReport(df, column, idx, term= ''):
    report = df[column][idx]
    report = re.sub(r'[\x0b\u25A1\u25A2]', ' ', report)
    report = re.sub(r'   ', '\n\n', report)
    report = re.sub(r'  ', '\n', report)
    print(df.shape)
    highlightText(report, term)

# Query

In [326]:
qstr = 'infarct(?:\sat|\sof|\slesion|\sand|\score)|infarcts(?:\sat|\sof|\slesion|\sand|\score)|infarction(?:\sat|\sof|\slesion|\sand|\score)|infarctions(?:\sat|\sof|\slesion|\sand|\score)|(?<!no\s)occlusion(?:\sat|\sof)'
keyword = '|pc-ASPECTS|infarct|hypodense|vertigo|dizziness|ataxia|disequilibrium|nystagmus|posterior circulation stroke|brain|#|MRI|MRA|CTP(?![A-Za-z])|CT perfusion|perfusion|CTA(?![A-Za-z])|XXX'

## Filter BRAIN capital letter 15673

In [327]:
df2 = df[df['Report'].str.contains('BRAIN')].reset_index(drop=True)
df2

,Id,Date,Report,Type
0,0,2022-07-13 02:57:59.247,EMERGENCY MDCT OF THE BRAIN HISTORY: Known c...,CT
1,1,2022-07-13 06:32:58.923,"MRI OF THE BRAIN \t History: U/D DM, HT, DLP,...",MRI
2,2,2022-07-13 06:35:10.970,MRI OF THE BRAIN AND HIPPOCAMPI (dementia prot...,MRI
3,4,2022-07-13 06:54:12.140,MDCT OF THE BRAIN HISTORY: F/U a of NPH S/P ...,CT
4,5,2022-07-13 07:30:26.893,MDCT OF THE BRAIN HISTORY: F/U subdural hemo...,CT
...,...,...,...,...
15668,19386,2023-07-17 12:09:10.360,MRI OF THE BRAIN HISTORY: Known case of rela...,MRI
15669,19387,2023-07-17 15:17:17.250,MDCT OF THE BRAIN HISTORY: Acute right PCA t...,CT
15670,19388,2023-07-17 13:33:05.590,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
15671,19388,2023-07-17 14:10:19.167,EMERGENCY MRI OF THE BRAIN (limited protocol f...,MRI


# Class Filter

### CT brain 7694 for pretrained model

In [328]:
df2[df2['Type'].str.contains('CT', case=False)].reset_index(drop=True)

,Id,Date,Report,Type
0,0,2022-07-13 02:57:59.247,EMERGENCY MDCT OF THE BRAIN HISTORY: Known c...,CT
1,4,2022-07-13 06:54:12.140,MDCT OF THE BRAIN HISTORY: F/U a of NPH S/P ...,CT
2,5,2022-07-13 07:30:26.893,MDCT OF THE BRAIN HISTORY: F/U subdural hemo...,CT
3,7,2022-07-13 10:19:48.400,MDCT OF THE BRAIN HISTORY: Known case of pro...,CT
4,11,2022-07-13 09:40:21.493,EMERGENCY MDCT OF THE BRAIN HISTORY: History ...,CT
...,...,...,...,...
7689,19380,2023-07-17 10:54:57.750,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
7690,19383,2023-07-17 10:10:39.900,MDCT OF THE BRAIN HISTORY: Known case astroc...,CT
7691,19387,2023-07-17 15:17:17.250,MDCT OF THE BRAIN HISTORY: Acute right PCA t...,CT
7692,19388,2023-07-17 13:33:05.590,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT


### CTP 109

In [329]:
df2[df2['Report'].str.contains('CT perfusion|CTP', case=False)].reset_index(drop=True)

,Id,Date,Report,Type
0,294,2022-07-19 10:04:42.163,CTA AND CO2 CHALLENGED CT PERFUSION OF THE BRA...,CT
1,315,2022-07-19 18:03:39.630,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
2,1891,2022-08-19 20:40:50.530,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
3,1891,2022-08-19 21:18:23.923,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
4,2153,2022-08-25 07:44:54.790,CTA AND CO2 CHALLENGED CT PERFUSION OF THE BRA...,CT
...,...,...,...,...
104,18272,2023-06-24 20:12:18.907,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
105,19159,2023-07-12 18:50:27.900,EMERGENCY MDCT OF THE BRAIN HISTORY: Known c...,CT
106,19192,2023-07-13 09:19:25.987,"MDCT, CTA AND CO2 CHALLENGED CT PERFUSION OF ...",CT
107,19196,2023-07-13 10:49:54.950,"MDCT, CTA AND CO2 CHALLENGED CT PERFUSION OF T...",CT


In [330]:
id = 315
df2[df2['Id']==id]['Report']
idx = df2[df2['Id']==id]['Report'].index[0]
printReport(df2,'Report',idx, qstr+keyword)
# df2[df2['Id']==315]['Report']
# printReport(df2,'Report',256, qstr+keyword)

(15673, 4)
EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NECK (Stroke protocol) WITH CT ADDITIONAL CT PERFUSION OF THE BRAIN

HISTORY: Known case of NPH s/p VP shunt with chronic subdural hematoma presented with left hemiparesis for 5 hours (last seen normal 13:00 on 19/07/2022). The physical examination revealed E4V5M6, left facial palsy, dysarthria and motor power grade II at left upper and lower extremities [NIHSS =13]. He was sent for rule out acute stroke.
TECHNIQUES: Plain, multiphase CTA and post contrast-enhanced axial helical scans of the brain and CTA of the neck were performed using 1.25-mm and 2.5 slice thicknesses with multiplanar reconstructions. Additional CT perfusion of the brain was also performed.
Intravenous contrast brand, concentration, volume: Visipaque320, 70ml.
Immediate complication: None.

COMPARISON: Prior CT of the brain from other hospital on 18/01/2022, MRI brain from other institute on 10/5/2018 and MRA/MRI of the brain on 13/4/2015. FINDINGS:
Cerebral par

### CTA 1032(978(CTA only))

In [331]:
df2[df2['Report'].str.contains('MDCT ANGIOGRAPHY', case=False) & ~df2['Report'].str.contains('CT perfusion|CTP', case=False)].reset_index(drop=True)

,Id,Date,Report,Type
0,12,2022-07-13 10:21:47.590,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
1,111,2022-07-15 02:39:53.027,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
2,112,2022-07-15 10:14:46.297,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
3,118,2022-07-15 23:03:07.830,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
4,128,2022-07-15 10:45:17.623,MDCT ANGIOGRAPHY OF THE BRAIN AND NECK HISTO...,CT
...,...,...,...,...
973,19314,2023-07-15 21:33:47.417,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
974,19320,2023-07-15 18:30:01.730,MDCT ANGIOGRAPHY OF THE BRAIN History: Case ...,CT
975,19358,2023-07-16 19:19:46.513,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
976,19380,2023-07-17 10:54:57.750,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT


In [332]:
df2[df2['Report'].str.contains('MDCT ANGIOGRAPHY', case=False) & df2['Report'].str.contains('occlusion(?:\sat|\sof)', case=False)].reset_index(drop=True)

,Id,Date,Report,Type
0,111,2022-07-15 02:39:53.027,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
1,243,2022-07-18 17:00:37.697,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
2,315,2022-07-19 18:03:39.630,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
3,343,2022-07-20 18:46:31.163,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN HIST...,CT
4,604,2022-07-25 08:36:21.790,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
...,...,...,...,...
192,18790,2023-07-05 17:55:05.960,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
193,18901,2023-07-07 19:00:14.243,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
194,19259,2023-07-14 07:47:34.357,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
195,19259,2023-07-14 08:24:54.800,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT


In [333]:
id = 111
df2[df2['Id']==id]['Report']
idx = df2[df2['Id']==id]['Report'].index[0]
printReport(df2,'Report',idx, qstr+keyword)

(15673, 4)
EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NECK (Stroke protocol)

HISTORY: A 67-year-old female; known case of HT, DM and DLP, presented with right hemiparesis for 2 hours (last seen normal 0:00). Her physical examination shows E4V5M6, right facial palsy, motor power grade 0 at right upper extremity, grade IV at left upper extremity and grade III at both lower extremities [NIHSS = 25]. She was sent to rule out acute stroke.
TECHNIQUES: Plain, multiphase CTA, and contrast-enhanced axial helical scans of the brain and neck were performed using 1.25-mm and 2.5-mm slice thicknesses with multiplanar reconstructions.
Intravenous contrast brand, concentration, volume: Optiray350, 70ml.
Immediate complication: None.
COMPARISON: None.

FINDINGS:
Brain:
Cerebral parenchyma: The study reveals age-appropriated brain volume. There is a focal area of loss grey-white differentiation at left parietal lobe. A well-defined hypodense areas with associated ex-vacuo dilatation of left occipita

### CT+MRI 24

Loop check CT and follow up MRI

In [334]:
countCT = 0
countMRI = 0
foundMRIList = []
for i in range(df2.shape[0]):
# for i in range(1000):
    # print(df2['Id'][i], df2['Type'][i])
    if(i>0):
        if(df2['Id'][i] != df2['Id'][i-1]):
            countCT = 0
            countMRI = 0    
    if(df2['Type'][i]=='CT'):
        countCT += 1
    if(df2['Type'][i]=='MRI'):
        countMRI += 1
    # print('Count : ',countCT, countMRI)
    if(countCT>=1 & countMRI>=1):
        foundMRIList.append(df2['Id'][i])
        # print('Found',df2['Id'][i])
    
print(len(foundMRIList), foundMRIList)

24 [109, 534, 552, 1884, 2342, 2668, 2807, 5008, 5402, 5783, 6196, 6208, 6891, 8173, 8443, 12081, 13948, 14385, 15938, 18011, 18116, 18343, 19033, 19388]


In [347]:
df2[df2['Id']==109]['Report']

92    EMERGENCY MDCT OF THE BRAIN   HISTORY: A 57-ye...
93    MRI OF THE BRAIN WITH MRA OF THE BRAIN AND NEC...
Name: Report, dtype: object

In [336]:
# df2[df2['Id']==1884]["Report"][2356]
printReport(df2,'Report',93, qstr+keyword)

(15673, 4)
MRI OF THE BRAIN WITH MRA OF THE BRAIN AND NECK

HISTORY: Known case of recently diagnosed acute stroke (12/07/2022) presented with severe dysarthria and left hemiparesis. His physical examination revealed motor power grade II of the left upper extremity and grade IV of the left lower extremity with left hemiparesthesia. He was sent for evaluation.
TECHNIQUES:
- Sagittal MDEFT T1W
- Axial 3D BrainVIEW T1W, THRIVE T1W FS+Gd, T2W FS, DWI with ADC mapping, SWI
- THRIVE T1W FS+Gd, 3D BrainVIEW T1W FS+Gd and FLAIR FS+Gd with multiplanar reconstructions
- TOF MRA and CE-MRA of the brain and neck

 COMPARISON: The prior CT of the brain on the same date (15:53).
FINDINGS: Limited comparison due to different study modalities.
Non-vascular
The current study again reveals diffuse brain volume loss. Redemonstration of acute to early subacute infarction seen as T2/FLAIR hyperintense lesion with restricted diffusion involving the body of the right caudate nucleus, right corona radiata, ri

### FU-NCCT 55

Loop check CT and FU-NCCT

In [337]:
countCT = 0
# countMRI = 0
foundFUNCCTList = []
for i in range(df2.shape[0]):
# for i in range(1000):
    # print(df2['Id'][i], df2['Type'][i])
    if(i>0):
        if(df2['Id'][i] != df2['Id'][i-1]):
            countCT = 0
    if(df2['Type'][i]=='CT'):
        countCT += 1
    # print('Count',i,':',countCT)
    if(countCT>=2 and (len(foundFUNCCTList) == 0 or foundFUNCCTList[-1]!=df2['Id'][i])):
        foundFUNCCTList.append(df2['Id'][i])
        # print('Found',df2['Id'][i])
    
print(len(foundFUNCCTList), foundFUNCCTList)

52 [772, 819, 1891, 2239, 2328, 2454, 2740, 3280, 3768, 3919, 4094, 4227, 4244, 4284, 4495, 5271, 5870, 6328, 6512, 6683, 7192, 7517, 7550, 8834, 9010, 9354, 9406, 9739, 10022, 10317, 10493, 10632, 11452, 12550, 12733, 13163, 13363, 13907, 13984, 14012, 14792, 15116, 15373, 15434, 15479, 15946, 16848, 16927, 18272, 18396, 18888, 19259]


In [338]:
df2[df2['Id']==foundFUNCCTList[0]]

,Id,Date,Report,Type
623,772,2022-07-29 01:59:01.817,EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NE...,CT
624,772,2022-07-29 03:01:06.213,EMERGENCY MDCT OF THE BRAIN HISTORY: Known c...,CT
625,772,2022-07-29 22:38:26.827,MDCT OF THE BRAIN HISTORY & INDICATION: 79-y...,CT


In [341]:
# df2[df2['Id']==1884]["Report"][2356]
printReport(df2,'Report',623, qstr+keyword)

(15673, 4)
EMERGENCY MDCT ANGIOGRAPHY OF THE BRAIN AND NECK (Stroke protocol)

HISTORY: Known case of left PICA infarction, DM and CKD, presented with right hemiparesis for 2 hours. Physical examination reveals motor power grade IV of the right extremities, dysarthria and conjugate eye deviation toward the right side (NIHSS = 6). CTA was requested to evaluate for stroke.
TECHNIQUES: Plain, multiphase CTA and contrast-enhanced axial helical scans of the brain and neck were performed using 1.25-mm and 2.5-mm slice thicknesses with multiplanar and MIP reconstructions.
Intravenous contrast brand, concentration, volume: Xenetix 350, 70ml.
Fasting (NPO) time: N/A
Immediate complication: None.

COMPARISON: MRI of the brain on 11/09/2021.
FINDINGS:
HEAD/BRAIN
Cerebral parenchyma: The study reveals diffuse brain volume loss, appropriate for age. There is a well-defined hypodense lesion involving the right caudate nucleus, anterior limb of the right internal capsule and right lentiform nucleus, 